# Data Cleaning & Preprocessing

This notebook performs:
1. Data Loading
2. Cleaning (Missing values, Invalid dimensions)
3. Feature Engineering (Volume calculation)
4. Material Mapping
5. Exporting processed data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np

# Try importing fuzzywuzzy, fallback to thefuzz if needed
try:
    from fuzzywuzzy import fuzz, process
except ImportError:
    try:
        from thefuzz import fuzz, process
    except ImportError:
        print("Warning: fuzzywuzzy or thefuzz not found. Install with: pip install fuzzywuzzy python-Levenshtein")
        # Fallback: simple substring matching only
        def process_extractOne(text, choices, scorer=None):
            # Simple fallback - just return first match or None
            for choice in choices:
                if text.lower() in choice.lower() or choice.lower() in text.lower():
                    return (choice, 100)
            return None
        process = type('obj', (object,), {'extractOne': process_extractOne})
        fuzz = type('obj', (object,), {'token_sort_ratio': lambda x, y: 0})

import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading

In [ ]:
# Load both CSV files
history_df = pd.read_csv('real_packaging_history (1).csv')
materials_df = pd.read_csv('materials_database_600 (1).csv')

print("History Data Shape:", history_df.shape)
print("Materials Data Shape:", materials_df.shape)
print("\nHistory Columns:", history_df.columns.tolist())
print("\nMaterials Columns:", materials_df.columns.tolist())
print("\nFirst few rows of history:")
history_df.head()

In [ ]:
# Check for missing values
print("Missing values in history data:")
print(history_df.isnull().sum())
print("\nMissing values in materials data:")
print(materials_df.isnull().sum())

In [ ]:
# Impute missing 'Cost_USD' and 'CO2_Emission_kg' with median of their respective 'Category'
for col in ['Cost_USD', 'CO2_Emission_kg']:
    if history_df[col].isnull().sum() > 0:
        category_medians = history_df.groupby('Category')[col].median()
        history_df[col] = history_df.apply(
            lambda row: category_medians[row['Category']] if pd.isnull(row[col]) else row[col],
            axis=1
        )
        print(f"Imputed {history_df[col].isnull().sum()} missing values in {col}")

print("\nMissing values after imputation:")
print(history_df[['Cost_USD', 'CO2_Emission_kg']].isnull().sum())

In [ ]:
# Fix invalid dimensions: If L_cm, W_cm, or H_cm are 0, replace with average for that Category
for dim in ['L_cm', 'W_cm', 'H_cm']:
    invalid_mask = (history_df[dim] == 0) | (history_df[dim].isnull())
    if invalid_mask.sum() > 0:
        category_means = history_df.groupby('Category')[dim].mean()
        history_df[dim] = history_df.apply(
            lambda row: category_means[row['Category']] if (row[dim] == 0 or pd.isnull(row[dim])) else row[dim],
            axis=1
        )
        print(f"Fixed {invalid_mask.sum()} invalid values in {dim}")

print("\nInvalid dimensions (zeros) after fixing:")
print((history_df[['L_cm', 'W_cm', 'H_cm']] == 0).sum())

In [ ]:
# Calculate Product_Volume_m3 as (L*W*H)/1,000,000
history_df['Product_Volume_m3'] = (history_df['L_cm'] * history_df['W_cm'] * history_df['H_cm']) / 1_000_000

print("Product_Volume_m3 calculated successfully!")
print(f"Volume statistics:\n{history_df['Product_Volume_m3'].describe()}")

## 2. Advanced Material Mapping

In [ ]:
# Get unique packaging names from history and material names from database
packaging_used = history_df['Packaging_Used'].unique()
material_names = materials_df['Material_Name'].unique()

print(f"Unique packaging types in history: {len(packaging_used)}")
print(f"Unique materials in database: {len(material_names)}")
print("\nSample packaging types:", packaging_used[:10])
print("\nSample material names:", material_names[:10])

In [ ]:
# Create a mapping function using fuzzy matching
def map_packaging_to_material(packaging_name, material_list, threshold=60):
    """
    Maps packaging name to material name using fuzzy matching.
    Also handles common substring matches.
    """
    # Direct substring matching for common cases
    packaging_lower = packaging_name.lower()
    
    # Common mappings
    if 'mushroom' in packaging_lower or 'mycelium' in packaging_lower:
        matches = [m for m in material_list if 'mushroom' in m.lower() or 'mycelium' in m.lower()]
        if matches:
            return matches[0]
    
    if 'wood' in packaging_lower or 'crate' in packaging_lower:
        matches = [m for m in material_list if 'plywood' in m.lower()]
        if matches:
            return matches[0]
    
    if 'kraft' in packaging_lower:
        matches = [m for m in material_list if 'kraft' in m.lower()]
        if matches:
            return matches[0]
    
    if 'pla' in packaging_lower or 'bioplastic' in packaging_lower:
        matches = [m for m in material_list if 'pla' in m.lower() or 'bioplastic' in m.lower()]
        if matches:
            return matches[0]
    
    if 'bubble' in packaging_lower or 'ldpe' in packaging_lower:
        matches = [m for m in material_list if 'bubble' in m.lower() or 'ldpe' in m.lower()]
        if matches:
            return matches[0]
    
    if 'pet' in packaging_lower and 'recycled' in packaging_lower:
        matches = [m for m in material_list if 'pet' in m.lower()]
        if matches:
            return matches[0]
    
    if 'honeycomb' in packaging_lower or 'paper' in packaging_lower:
        matches = [m for m in material_list if 'paper' in m.lower() or 'honeycomb' in m.lower()]
        if matches:
            return matches[0]
    
    # Additional mappings for other packaging types
    if 'corrugated' in packaging_lower or 'cardboard' in packaging_lower:
        matches = [m for m in material_list if 'cardboard' in m.lower() or 'corrugated' in m.lower()]
        if matches:
            return matches[0]
    
    if 'styrofoam' in packaging_lower or 'eps' in packaging_lower:
        matches = [m for m in material_list if 'foam' in m.lower() and 'polyurethane' in m.lower()]
        if matches:
            return matches[0]
    
    if 'cornstarch' in packaging_lower:
        matches = [m for m in material_list if 'cornstarch' in m.lower()]
        if matches:
            return matches[0]
    
    # Fuzzy matching as fallback
    try:
        best_match = process.extractOne(packaging_name, material_list, scorer=fuzz.token_sort_ratio)
        if best_match and best_match[1] >= threshold:
            return best_match[0]
    except:
        # If fuzzy matching fails, try simple substring match
        for material in material_list:
            if packaging_lower in material.lower() or material.lower() in packaging_lower:
                return material
    
    return None

# Create mapping dictionary
packaging_to_material = {}
for packaging in packaging_used:
    mapped = map_packaging_to_material(packaging, material_names)
    packaging_to_material[packaging] = mapped
    if mapped:
        print(f"'{packaging}' -> '{mapped}'")
    else:
        print(f"'{packaging}' -> NOT FOUND")

print(f"\nSuccessfully mapped {sum(1 for v in packaging_to_material.values() if v is not None)}/{len(packaging_to_material)} packaging types")

In [ ]:
# Apply mapping to history dataframe
history_df['Material_Name'] = history_df['Packaging_Used'].map(packaging_to_material)

# Check how many were successfully mapped
print(f"Mapped rows: {history_df['Material_Name'].notna().sum()}/{len(history_df)}")
print(f"Unmapped rows: {history_df['Material_Name'].isna().sum()}")

# Show unmapped packaging types
if history_df['Material_Name'].isna().sum() > 0:
    unmapped = history_df[history_df['Material_Name'].isna()]['Packaging_Used'].unique()
    print(f"\nUnmapped packaging types: {unmapped}")

In [ ]:
# Merge material properties from materials database
# Rename CO2_Emission_kg to Material_CO2_Factor in materials_df for clarity
materials_merge = materials_df[['Material_Name', 'Density_kg_m3', 'Cost_per_kg', 'CO2_Emission_kg']].copy()
materials_merge = materials_merge.rename(columns={'CO2_Emission_kg': 'Material_CO2_Factor'})

# Merge with history dataframe
history_df = history_df.merge(
    materials_merge,
    on='Material_Name',
    how='left',
    suffixes=('', '_material')
)

# Rename Density column for clarity
history_df = history_df.rename(columns={'Density_kg_m3': 'Material_Density'})

print("Merged material properties:")
print(f"Rows with material properties: {history_df['Material_Density'].notna().sum()}/{len(history_df)}")
print("\nSample merged data:")
history_df[['Packaging_Used', 'Material_Name', 'Material_Density', 'Material_CO2_Factor', 'Cost_per_kg']].head(10)

In [ ]:
# Remove rows where material mapping failed (no material properties)
initial_rows = len(history_df)
history_df = history_df[history_df['Material_Density'].notna()].copy()
removed_rows = initial_rows - len(history_df)

print(f"Removed {removed_rows} rows without material mapping")
print(f"Final dataset size: {len(history_df)} rows")

## 3. Export Processed Data

In [ ]:
# Save cleaned and processed data to CSV
output_file = 'cleaned_data.csv'
history_df.to_csv(output_file, index=False)
print(f"✅ Successfully saved cleaned data to {output_file}")